In [138]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py37_0         148 KB  conda-forge
    wheel-0.33.4               |           py37_0          34 KB  conda-forge
    xz-5.2.4                   |    h14c3975_1001         366 KB  conda-forge
    zlib-1.2.11                |    h14c3975_1004         101 KB  conda-forge
    python-3.7.3               |       h33d41f4_1        36.0 MB  conda-forge
    libffi-3.2.1               |    he1b5a44_1006          46 KB  conda-forge
    pip-19.1.1                 |           py37_0         1.8 MB  conda-forge
    sqlite-3.28.0              |       hcee41ef_1         1.9 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    readline-8.0 

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:38: UserWarning: No module named 'multiprocessing.synchronize'.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


ImportError: cannot import name 'synchronize'

### The next two cells are just to set up the creation of the table. I get the source of the table using the requests package and then just get the headings.

In [38]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [39]:
mytable = soup.find('table')
ths = mytable.find_all('th')
headings = [th.text.strip() for th in ths]
print(headings)

['Postcode', 'Borough', 'Neighbourhood']


### First, I get all of the text by stripping it out of the HTML.
### Then, I initialize some lists and fill them with values depending on what the index is, since the "entries" has a whole row in groups of 3. If any of the post codes and borough pairs are the same, then I append the second neighborhood onto the first and remove the second's row.
### Once I have the lists, I create a data frame and then filter it to remove any Borough that is Not Assigned.

In [89]:
tds = mytable.find_all('td')
entries = [td.text.strip() for td in tds]
post = []
borough = []
neighborhood = []
for i in range(0,len(entries)):
    if i%3 == 0:
               post.append(entries[i])
    elif i%3 == 1:
               borough.append(entries[i])
    elif i%3 == 2:
               neighborhood.append(entries[i])

for i in range(0,len(post)):
    for j in range(0,len(post)):
        try:
            if post[i] == post[j] and borough[i] == borough[j]:
                neighborhood[i] = neighborhood[i] + ", " + neighborhood[j]
                del post[j]
                del borough[j]
                del neighborhood[j]
        except: pass

wiki = pd.DataFrame({'Postcode':post,'Borough':borough,'Neighbourhood':neighborhood})
wiki = wiki[wiki["Borough"] != "Not assigned"]
wiki.head(10)

,Postcode,Borough,Neighbourhood
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,Lawrence Manor
5,M1B,Scarborough,"Rouge, Malvern"
6,M3B,North York,Don Mills North
7,M4B,East York,Parkview Hill
8,M5B,Downtown Toronto,Garden District
10,M9B,Etobicoke,"Martin Grove, Cloverdale, Islington, Princess ..."
11,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill"
12,M3C,North York,"Flemingdon Park, Don Mills South"


In [50]:
wiki.shape

(74, 3)

In [90]:
geocodes = pd.read_csv('https://cocl.us/Geospatial_data')

### I decided to use the CSV since the geocodes package was not working. Below, I did a join by using the post codes as indices, then bringing it back as a named column

In [134]:
wikijoin = wiki.set_index("Postcode")
wikijoin = wikijoin.join(geocodes.set_index("Postal Code"))
wikijoin = wikijoin.reset_index()
wikijoin.rename(columns={"index":"Postcode"}, inplace=True)
wikijoin.head(60)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711
3,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
4,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
5,M1K,Scarborough,"Kennedy Park, East Birchmount Park, Ionview",43.727929,-79.262029
6,M1L,Scarborough,"Oakridge, Clairlea, Golden Mile",43.711112,-79.284577
7,M1M,Scarborough,"Scarborough Village West, Cliffcrest, Cliffside",43.716316,-79.239476
8,M1N,Scarborough,Cliffside West,43.692657,-79.264848
9,M1P,Scarborough,"Wexford Heights, Dorset Park, Scarborough Town...",43.757410,-79.273304


In [137]:
import re
toronto = wikijoin[wikijoin["Borough"].isin(["West Toronto", "East Toronto","Central Toronto","Downtown Toronto"])]
toronto.shape

(31, 5)

In [139]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


# From here on down, there are so many problems with packages and certificates. I could not solve those, so the code will not resolve properly. If everything was proper, then this code should work

In [150]:
!conda install -c conda-forge folium
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.9.1               |             py_0          59 KB  conda-forge

The following packages will be UPDATED:

    folium: 0.5.0-py_0 conda-forge --> 0.9.1-py_0 conda-forge


folium-0.9.1         | 59 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


ModuleNotFoundError: No module named 'folium'

In [151]:
import folium

ModuleNotFoundError: No module named 'folium'

In [152]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

NameError: name 'folium' is not defined

In [174]:
{
    "tags": [
        "hide_input",
    ]
}
CLIENT_ID = "4FIZST1KXSBIHXCC4RSGU0QEOQVT5STP1ERXJ2UYQEELSBTL"
CLIENT_SECRET = "IVUBOCIUPWIOJBNF2PJ4PAJEKFHFT2GANNNG2ZFAWSMWKJD2"

In [176]:
radius = 500
LIMIT = 100
#CLIENT_ID = defined in hidden cell
#CLIENT_SECRET = defined in hidden cell
VERSION = '20190707'

def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [162]:

toronto_venues = getNearbyVenues(names=toronto['Neighbourhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )



Riverdale


OSError: Could not find a suitable TLS CA certificate bundle, invalid path: /opt/conda/envs/Python36/lib/python3.6/site-packages/certifi/cacert.pem

In [164]:
toronto_venues.groupby('Neighborhood').count()
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

NameError: name 'toronto_venues' is not defined

In [165]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

NameError: name 'toronto_onehot' is not defined

In [166]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

NameError: name 'toronto_grouped' is not defined

In [167]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [169]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

NameError: name 'toronto_grouped' is not defined

In [171]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

NameError: name 'toronto_grouped' is not defined

In [172]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

NameError: name 'kmeans' is not defined

In [173]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'folium' is not defined